In [6]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
sys.path.append("/home/pervinco/BKAI_MetaPolyp")

import cv2
import yaml
import numpy as np

from glob import glob
from data.batch_preprocess import load_img_mask, encode_mask

In [7]:
with open("/home/pervinco/BKAI_MetaPolyp/config.yaml", "r") as f:
    config = yaml.safe_load(f)

data_dir = config["data_dir"]
image_dir = f"{data_dir}/train"
mask_dir = f"{data_dir}/train_gt"

image_files = sorted(glob(f"{image_dir}/*"))
mask_files = sorted(glob(f"{mask_dir}/*"))


In [8]:
def get_bounding_boxes_for_classes(encoded_mask):
    classes = [1, 2]  # Defined classes in the mask
    bounding_boxes_dict = {}

    for cls in classes:
        # Create a binary mask for the current class
        binary_mask = (encoded_mask == cls).astype(np.uint8)

        # Find contours for the binary mask
        contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        bounding_boxes = []
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            xmin = x
            ymin = y
            xmax = x + w
            ymax = y + h
            bounding_boxes.append((xmin, ymin, xmax, ymax))
        
        bounding_boxes_dict[cls] = bounding_boxes
    
    return bounding_boxes_dict

In [9]:
if not os.path.isdir("./img_with_bboxes"):
    os.makedirs("./img_with_bboxes")

if not os.path.isdir(f"{data_dir}/bounding_boxes"):
    os.makedirs(f"{data_dir}/bounding_boxes")

In [10]:
for image_file, mask_file in zip(image_files, mask_files):
    file_name = image_file.split('/')[-1].split('.')[0]
    image, mask = load_img_mask(image_file, mask_file, size=config["img_size"])

    encoded_mask = encode_mask(mask)
    bounding_boxes = get_bounding_boxes_for_classes(encoded_mask)
    
    with open(f"{data_dir}/bounding_boxes/{file_name}.txt", "w") as f:
        for cls, bboxes in bounding_boxes.items():
            for bbox in bboxes:
                xmin, ymin, xmax, ymax = bbox[0], bbox[1], bbox[2], bbox[3]
                f.write(f"{cls},{xmin},{ymin},{xmax},{ymax}\n")
                # image = cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)

    # mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
    # overlay = cv2.addWeighted(image, 0.7, mask, 0.3, 0)
    # result = np.hstack((image, mask, overlay))

    # result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
    # cv2.imwrite(f"./img_with_bboxes/{file_name}.png", result)